<a href="https://colab.research.google.com/github/hussamalafandi/Generative_AI/blob/main/notebooks/08/08_training_and_fine-tuning_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From Pre-training to Fine-tuning: Practical Guide for Transformers

## Learning Objectives:

By the end of this notebook, students will:

* Understand the differences and purposes of pre-training, supervised fine-tuning (SFT), and preference-based training (DPO).
* Be able to fine-tune small, efficient transformer models (SmolLM2-135M) on practical datasets.
* Evaluate fine-tuned models quantitatively (perplexity/loss) and qualitatively (generation quality).
* Get introduced to high-performance libraries (Unsloth) for efficient transformer fine-tuning.

## Introduction and Concepts

<div style="text-align: center;">
    <img src="https://images.ctfassets.net/kftzwdyauwt9/40in10B8KtAGrQvwRv5cop/8241bb17c283dced48ea034a41d7464a/chatgpt_diagram_light.png?w=2048&q=80&fm=webp" alt="ChatGPT training phases" width="1200" />
</div>

Image source: [openai.com](https://openai.com/index/chatgpt/)

### What is Pre-training?

**Pre-training** is a process in which language models are initially trained on large-scale datasets using **self-supervised learning**. In simple terms, these models learn directly from the text itself without explicit labels provided by humans.

Common pre-training tasks include:

* **Causal Language Modeling (CLM):** Predicting the next word/token based on previous context.

* **Masked Language Modeling (MLM):** Predicting hidden (masked) words in a sentence (e.g., used by BERT).

Through pre-training, models capture fundamental language understanding, grammar, reasoning, and context comprehension. This general knowledge becomes the basis for further specialization through fine-tuning.

#### SmolLM2 (Small and Efficient Model)

In this course, we’ll use SmolLM2-135M, a compact and efficient transformer-based language model developed specifically to offer robust performance on modest hardware resources. SmolLM2 balances capability and efficiency, making it ideal for educational purposes, prototyping, and running on limited hardware (e.g., GPUs available via Colab).

### What is Supervised Fine-Tuning (SFT)?

Although pre-trained models have broad language capabilities, they often lack task-specific accuracy. **Supervised Fine-Tuning (SFT)** bridges this gap, refining the general knowledge learned during pre-training by training the model further on task-specific data with explicit labels or prompts.

Through fine-tuning, models become adept at specialized tasks, such as:

* Conversational assistants
* Text summarization
* Sentiment analysis
* Domain-specific language generation

#### SmolTalk Dataset

In this notebook, we'll practically perform supervised fine-tuning using **SmolTalk**, a compact conversational dataset designed specifically for fine-tuning lightweight language models. SmolTalk is carefully crafted to provide realistic conversational exchanges, helping our SmolLM2 model improve significantly in generating natural dialogues and responding to instructions.

### Preference-based Fine-tuning (DPO)

Beyond supervised fine-tuning, recent approaches also incorporate **Preference-based Fine-tuning (DPO, Direct Preference Optimization)**. Instead of optimizing purely for task accuracy or next-token prediction, DPO fine-tunes models based on human-generated feedback indicating preference for specific responses.

This method ensures models not only become task-specific but also align better with human preferences, improving their real-world usability, helpfulness, and alignment with ethical guidelines.

#### UltraFeedback Dataset (Introduction)

Later in our course, we will explore DPO practically using the **UltraFeedback dataset**, which contains numerous examples of human preferences ranking model-generated outputs. UltraFeedback helps models like SmolLM2 adapt to human-preferred conversational styles and improves their ability to generate helpful, appropriate, and aligned responses.



<div style="text-align: center;">
    <img src="https://cdn-uploads.huggingface.co/production/uploads/61c141342aac764ce1654e43/RvHjdlRT5gGQt5mJuhXH9.png" alt="SmolLM2 Ecosystem" width="1200" />
</div>

Image source: [huggingface.co](https://huggingface.co/HuggingFaceTB)

## Setup and Dataset Preparation

In this section, you'll set up your environment by installing essential libraries and exploring the dataset (**SmolTalk**) we'll use for supervised fine-tuning (SFT).

In [1]:
!pip install transformers datasets trl torch torchvision huggingface_hub evaluate

Load the dataset

In [2]:
from datasets import load_dataset
from pprint import pprint

train_dataset = load_dataset("HuggingFaceTB/smoltalk", "smol-rewrite", split="train")
eval_dataset = load_dataset("HuggingFaceTB/smoltalk", "smol-rewrite", split="test")

pprint(train_dataset[0]['messages'])

[{'content': "You're an AI assistant for text re-writing. Rewrite the input "
             'text to make it more concise while preserving its core meaning.',
  'role': 'system'},
 {'content': 'Hey Alex,\n'
             '\n'
             "I hope you're doing well! It's been a while since we met at the "
             'film festival last year. I was the one with the short film about '
             "the old abandoned factory. Anyway, I'm reaching out because I'm "
             'currently working on my thesis film project and I could really '
             'use some advice on cinematography. I remember our conversation '
             'about visual storytelling and I was hoping you might have some '
             'tips or insights to share.\n'
             '\n'
             'My film is a drama set in a small town, and I want to capture '
             'the mood and atmosphere of the location through my '
             "cinematography. I'm planning to shoot on location next month, "
             

Load **SmolLM2** Tokenizer

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M-Instruct")

The loaded dataset contains conversations with distinct roles, which need to be formatted into a consistent text sequence before being fed into the language model.

A **chat template** defines the structure for this formatting, ensuring uniformity in how roles are represented. Many tokenizers, including the SmolLM2 tokenizer, provide built-in methods for applying such templates.

The SmolLM2 tokenizer includes a chat template specifically designed to align with its conversational style.

In [4]:
# Using tokenizer's apply_chat_template method (if provided)
# We'll format the example as an instruction-response pair
chat_example = train_dataset[0]['messages']

# Check the default chat template (optional, but useful for inspection)
formatted_input = tokenizer.apply_chat_template(chat_example, tokenize=False)
print("\nFormatted input with chat template:\n", formatted_input)



Formatted input with chat template:
 <|im_start|>system
You're an AI assistant for text re-writing. Rewrite the input text to make it more concise while preserving its core meaning.<|im_end|>
<|im_start|>user
Hey Alex,

I hope you're doing well! It's been a while since we met at the film festival last year. I was the one with the short film about the old abandoned factory. Anyway, I'm reaching out because I'm currently working on my thesis film project and I could really use some advice on cinematography. I remember our conversation about visual storytelling and I was hoping you might have some tips or insights to share.

My film is a drama set in a small town, and I want to capture the mood and atmosphere of the location through my cinematography. I'm planning to shoot on location next month, but I'm still trying to figure out the best way to approach it. If you have any suggestions or resources you could point me to, I would be incredibly grateful.

Also, I heard from a mutual frien

Now, let's tokenize our dataset efficiently using the chat template. We'll define a tokenization function and apply it to the dataset:

In [5]:
def tokenize_with_chat_template(example):
    # Use the tokenizer's apply_chat_template method to format the input
    formatted_input = tokenizer.apply_chat_template(example['messages'], tokenize=False)

    # Tokenize the formatted input
    tokenized_input = tokenizer(formatted_input, truncation=True, padding="max_length")

    return tokenized_input

# Tokenize the entire dataset using the custom function
tokenized_dataset = train_dataset.map(
    tokenize_with_chat_template,
    batched=True,
    remove_columns=train_dataset.column_names
)

Map:   0%|          | 0/53342 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
tokenized_dataset

In [ ]:
print(tokenized_dataset[0]['input_ids'])

In [ ]:
# Decode back for sanity check
decoded_sample = tokenizer.decode(tokenized_dataset[0]['input_ids'])
print("\nDecoded sample:\n", decoded_sample)


In [6]:
from transformers import AutoModelForCausalLM
from trl import SFTTrainer, SFTConfig

model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-135M")


In [ ]:
!pip install bert_score sentence_transformers evaluate scikit-learn rouge_score

In [21]:
import evaluate
import numpy as np
import bert_score
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load necessary models and metrics
# metric_bleu = evaluate.load("bleu")
# metric_rouge = evaluate.load("rouge")
# sentence_model = SentenceTransformer('all-mpnet-base-v2')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Decode predictions and labels to text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Calculate BLEU and ROUGE
    # bleu_result = metric_bleu.compute(predictions=decoded_preds, references=decoded_labels)
    # rouge_result = metric_rouge.compute(predictions=decoded_preds, references=decoded_labels)

    # Calculate BERTScore
    P, R, F1 = bert_score.score(decoded_preds, decoded_labels, model_type="bert-base-uncased")

    # Calculate cosine similarity using sentence embeddings
    # original_embeddings = sentence_model.encode(decoded_labels)
    # rewritten_embeddings = sentence_model.encode(decoded_preds)
    # similarity_scores = cosine_similarity(original_embeddings, rewritten_embeddings)

    # Combine results into a dictionary
    result = {
        # "bleu": bleu_result["bleu"],
        # "rouge1": rouge_result["rouge1"],
        # "rouge2": rouge_result["rouge2"],
        # "rougeL": rouge_result["rougeL"],
        # "rougeLsum": rouge_result["rougeLsum"],
        "bertscore": F1.mean().item(),
        # "avg_cosine_similarity": similarity_scores.diagonal().mean()
    }

    return result


In [22]:
training_args = SFTConfig(
    output_dir="./smollm2-sft-results",   # Output directory for model checkpoints
    num_train_epochs=3,                   # Number of epochs
    learning_rate=3e-5,                   # Learning rate # try with 5e-5
    logging_steps=10,                     # How often to log training progress
    save_steps=100,                       # How often to save checkpoints
    per_device_train_batch_size=4,        # Set according to your GPU memory capacity
    per_device_eval_batch_size=8,
    eval_strategy="steps",                # Evaluate the model at regular intervals
    eval_steps=50,                        # Frequency of evaluation
    # gradient_accumulation_steps=4,
    # gradient_checkpointing=True,
)




In [ ]:
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    # compute_metrics=compute_metrics
)

trainer.train()


Step,Training Loss,Validation Loss
50,0.978800,1.319638
100,1.197300,1.244419
150,1.180600,1.215166
200,1.174200,1.196922
